In [1]:
# Configure the parent path to be the proj folder
import sys, os, torch, time
sys.path.append('../../')

# Import the block classes
from rwkv.block.v6_gold_finch.gold_finch_time_mix import GoldFinchTimeMix

# File to load
MODEL_FILENAME="L12-D768-x060b2_goco-minipile.pth"

# Run device, and run dtype to use
RUN_DEVICE="cpu"
RUN_DTYPE=torch.bfloat16

# Check for cuda device
if torch.cuda.is_available():
    RUN_DEVICE="cuda:0"

# Check if the reference weights exists
assert os.path.exists(f"./.model/{MODEL_FILENAME}"), "The reference weights does not exist. Please download it first (00-model-download.ipynb)"

# Loads the model weights
model_weight = torch.load(f"./.model/{MODEL_FILENAME}", map_location='cpu', weights_only=True, mmap=True)

# Model filename
print(f"### Model filename: {MODEL_FILENAME}")

# Lets get the n_dim, and setup the test module
n_dim = model_weight['emb.weight'].shape[1]
print(f"### Model n_dim: {n_dim}")

# List the model weights keys, and their shapes
print(f"### model weights keys:")
for key in model_weight:
    print(f"{key}: {model_weight[key].shape} - {model_weight[key].dtype}")

/home/picocreator/anaconda3/envs/rwkv-block/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/picocreator/anaconda3/envs/rwkv-block/lib/python3.11/site-packages/fla/layers/attn.py:23: UserWarning: Flash Attention is not installed. Please install it via `pip install flash-attn --no-build-isolation`
  warnings.warn("Flash Attention is not installed. Please install it via `pip install flash-attn --no-build-isolation`")


### Model filename: L12-D768-x060b2_goco-minipile.pth
### Model n_dim: 768
### model weights keys:
time_maa_x: torch.Size([1, 1, 768]) - torch.bfloat16
time_maa_token: torch.Size([1, 1, 768]) - torch.bfloat16
time_maa_w1: torch.Size([768, 32]) - torch.bfloat16
time_maa_w2: torch.Size([32, 768]) - torch.bfloat16
emb.weight: torch.Size([65536, 768]) - torch.bfloat16
blocks.0.ln1.weight: torch.Size([768]) - torch.bfloat16
blocks.0.ln1.bias: torch.Size([768]) - torch.bfloat16
blocks.0.ln2.weight: torch.Size([768]) - torch.bfloat16
blocks.0.ln2.bias: torch.Size([768]) - torch.bfloat16
blocks.0.ln0.weight: torch.Size([768]) - torch.bfloat16
blocks.0.ln0.bias: torch.Size([768]) - torch.bfloat16
blocks.0.att.time_maa_x: torch.Size([1, 1, 768]) - torch.bfloat16
blocks.0.att.time_maa_r: torch.Size([1, 1, 768]) - torch.bfloat16
blocks.0.att.time_maa_k: torch.Size([1, 1, 768]) - torch.bfloat16
blocks.0.att.time_maa_v: torch.Size([1, 1, 768]) - torch.bfloat16
blocks.0.att.time_maa_w: torch.Size([1,

In [2]:
# Initialize the channelmix state, and x state to test
x_state_0 = torch.ones(1, 1, n_dim, device=RUN_DEVICE, dtype=RUN_DTYPE)
x_state_1 = torch.ones(1, 1, n_dim, device=RUN_DEVICE, dtype=RUN_DTYPE)
tmix_shift_0 = torch.ones(1, n_dim, device=RUN_DEVICE, dtype=RUN_DTYPE)
tmix_shift_1 = torch.ones(1, n_dim, device=RUN_DEVICE, dtype=RUN_DTYPE)
tmix_wkv_0 = torch.ones(1, n_dim // 64, 64, 64, device=RUN_DEVICE, dtype=RUN_DTYPE)
tmix_wkv_1 = torch.ones(1, n_dim // 64, 64, 64, device=RUN_DEVICE, dtype=RUN_DTYPE)

# Build the cmix block
gf_tmix = GoldFinchTimeMix({ "n_layer":24, "n_dim":n_dim, "layer_id":0, "device":RUN_DEVICE, "dtype":RUN_DTYPE })
gf_tmix.load_from_model_state_dict(model_weight, 0)

# Log each item shape
gf_tmix_state = gf_tmix.state_dict()
print(f"### gftmix state keys:")
for key in gf_tmix_state:
    print(f"tmix.{key}: {gf_tmix_state[key].shape} - {gf_tmix_state[key].dtype}")
print("----")

### gftmix state keys:
tmix.time_maa_x: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.time_maa_r: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.time_maa_k: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.time_maa_v: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.time_maa_w: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.time_maa_v2: torch.Size([1, 1, 768]) - torch.bfloat16
tmix.receptance.weight: torch.Size([768, 768]) - torch.bfloat16
tmix.key.weight: torch.Size([768, 768]) - torch.bfloat16
tmix.value.weight: torch.Size([768, 768]) - torch.bfloat16
tmix.output.weight: torch.Size([768, 768]) - torch.bfloat16
tmix.ln_x.weight: torch.Size([768]) - torch.bfloat16
tmix.ln_x.bias: torch.Size([768]) - torch.bfloat16
----


In [3]:
# Iteration to test
TEST_STEPS = 1000

### TMix
with torch.inference_mode():

    # This is a warmup
    t0 = time.time()
    out_x = x_state_0
    t_shift = tmix_shift_0
    t_wkv = tmix_wkv_0
    for i in range(TEST_STEPS):
        out_x, t_shift, t_wkv = gf_tmix(x_state_1, t_shift, tmix_wkv_1)

    # The actual run
    t1 = time.time()
    out_x = x_state_0
    t_shift = tmix_shift_0
    t_wkv = tmix_wkv_0
    for i in range(TEST_STEPS):
        out_x, t_shift, t_wkv = gf_tmix(x_state_1, t_shift, tmix_wkv_1)
    t2 = time.time()
    print(f'1 tmix forward passes (normal): {(t2-t1)*1000/TEST_STEPS} ms ({RUN_DEVICE}, {RUN_DTYPE})')


1 tmix forward passes (normal): 0.857581377029419 ms (cuda:0, torch.bfloat16)


In [5]:
# Iteration to test
TEST_STEPS = 1000

### TMix
with torch.inference_mode():

    # This is a warmup
    t0 = time.time()
    out_x = x_state_0
    t_shift = tmix_shift_0
    t_wkv = tmix_wkv_0
    for i in range(TEST_STEPS):
        out_x, t_shift, t_wkv = gf_tmix.forward_with_compile(x_state_1, t_shift, tmix_wkv_1, out_x, t_shift, t_wkv)

    # The actual run
    t1 = time.time()
    out_x = x_state_0
    t_shift = tmix_shift_0
    t_wkv = tmix_wkv_0
    for i in range(TEST_STEPS):
        out_x, t_shift, t_wkv = gf_tmix.forward_with_compile(x_state_1, t_shift, tmix_wkv_1, out_x, t_shift, t_wkv)
    t2 = time.time()
    print(f'1 tmix forward passes (compiled): {(t2-t1)*1000/TEST_STEPS} ms ({RUN_DEVICE}, {RUN_DTYPE})')


1 tmix forward passes (compiled): 0.6740646362304688 ms (cuda:0, torch.bfloat16)
